In [65]:
import pandas as pd
import keras
from keras.layers import Conv1D
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Flatten
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import LSTM

In [35]:
train = pd.read_csv('../data/acc_data/training_large.csv')#,header = None)
test = pd.read_csv('../data/acc_data/validation_large.csv')#,header = None)

In [36]:
train.head()

,vx,vy,vz,dx,dy,vfx,vfy,vfz,afx,afy,...,num_v_labels,ax,ay,az,vx_lag_1,vy_lag_1,vz_lag_1,vx_lag_2,vy_lag_2,vz_lag_2
0,-10.738884,-2.270775,-0.017484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,-0.370861,0.324686,0.063559,-10.776157,-2.238142,-0.011096,-10.822652,-2.210152,-0.011221
1,-10.702093,-2.299289,-0.027242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,-0.366060,0.283704,0.097093,-10.738884,-2.270775,-0.017484,-10.776157,-2.238142,-0.011096
2,-10.649046,-2.326863,-0.015441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,-0.527806,0.274360,-0.117417,-10.702093,-2.299289,-0.027242,-10.738884,-2.270775,-0.017484
3,-10.598294,-2.345125,-0.018072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,-0.504967,0.181704,0.026169,-10.649046,-2.326863,-0.015441,-10.702093,-2.299289,-0.027242
4,-10.558764,-2.369425,-0.021834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,-0.393321,0.241773,0.037435,-10.598294,-2.345125,-0.018072,-10.649046,-2.326863,-0.015441


In [75]:
X = train.loc[:,['vx', 'vy', 'vz', 'dx', 'dy', 'vfx', 'vfy', 'vfz', 'afx', 'afy', 'afz', 'num_v_labels',
                 'vx_lag_1','vy_lag_1','vz_lag_1','vx_lag_2','vy_lag_2','vz_lag_2']]
y = train.loc[:,['ax','ay']].values 
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_val = test.loc[:,['vx', 'vy', 'vz', 'dx', 'dy', 'vfx', 'vfy', 'vfz', 'afx', 'afy', 'afz', 'num_v_labels',
                    'vx_lag_1','vy_lag_1','vz_lag_1','vx_lag_2','vy_lag_2','vz_lag_2']]
y_val = test.loc[:, ['ax','ay']].values

##### Feedfordward NN

In [59]:
model = Sequential()
model.add(Dense(64, input_dim=18, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
epochs = 10
lrate = 0.01
decay = lrate/epochs
model.compile(loss='mean_absolute_error', optimizer='adam', metrics = ['mae'])
print(model.summary())

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 64)                1216      
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_67 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_26 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_69 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_70 (Dense)             (None, 8)               

In [60]:
model.fit(X_train,y_train,epochs = epochs)

Epoch 1/10
115778/115778 [==============================] - 4s 39us/step - loss: 0.3627 - mae: 0.3627
Epoch 2/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3616 - mae: 0.3616
Epoch 3/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3563 - mae: 0.3563
Epoch 4/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3526 - mae: 0.3526
Epoch 5/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3489 - mae: 0.3489
Epoch 6/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3431 - mae: 0.3431
Epoch 7/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3398 - mae: 0.3398
Epoch 8/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3370 - mae: 0.3370
Epoch 9/10
115778/115778 [==============================] - 4s 37us/step - loss: 0.3340 - mae: 0.3340
Epoch 10/10
115778/115778 [==============================] - 4s 38us/step - loss: 

In [61]:
_, mae = model.evaluate(X_test, y_test)
print('MAE: %.2f' % (mae))

38593/38593 [==============================] - 1s 16us/step
MAE: 0.32


In [63]:
y_pred = model.predict(X_val)

In [66]:
print("MAE:", mean_absolute_error(y_val,y_pred))

MAE: 0.3532543465275578


##### LSTM NN (Not ready)

In [77]:
model = keras.Sequential()
#model.add(Dense(128, input_dim=18, activation='relu'))
model.add(LSTM(128,return_sequences=True, input_dim = 18, dropout=0.25,recurrent_dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(LSTM(32, return_sequences=True, dropout=0.25,recurrent_dropout=0.1))
model.add(LSTM(16, return_sequences=False, dropout=0.25,recurrent_dropout=0.1))
model.add(Dense(2, activation='sigmoid'))
epochs = 10
lrate = 0.01
decay = lrate/epochs
model.compile(loss='mean_absolute_error', optimizer='adam', metrics = ['mae'])
print(model.summary())

C:\Users\19296\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\19296\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, return_sequences=True, dropout=0.25, recurrent_dropout=0.1, input_shape=(None, 18))`
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, None, 128)         75264     
_________________________________________________________________
dense_78 (Dense)             (None, None, 64)          8256      
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 32)          12416     
_________________________________________________________________
lstm_5 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_79 (Dense)             (None, 2)                 34        
Total params: 99,106
Trainable params: 99,106
Non-trainable params: 0
_________________________________________________________________
None
